# Mergeall


In [1]:
import sys
import os
import json
from tqdm import tqdm
import pandas as pd
import seaborn as sns
import glob
import matplotlib.pyplot as plt
import numpy as np
import datetime

import warnings
warnings.filterwarnings("ignore")

sns.set_theme(context='notebook', style='whitegrid')
pd.set_option("display.max_rows", 100)

In [2]:
start_run = datetime.datetime.now()

with open("prefix.txt", "r") as f:
    prefix = f.read()
prefix

'housing_prompt_v1'

In [ ]:
# ZIP large data files, so they can be pushed to GitHub -- convert to code cell to run
!find . -type f -name "*.jsonl" ! -name "*.jsonl.zip" -exec sh -c 'zip "{}.zip" "{}"' \;
!find . -type f -name "*.log" ! -name "*.log.zip" -exec sh -c 'zip "{}.zip" "{}"' \;

# Data Cleaning

In [ ]:
fns = glob.glob(f"input_data/batch_results/{prefix}*.jsonl")
fns.sort()
print(fns, len(fns))

In [ ]:
dfs = []
for file in tqdm(fns):
    print(file)
    df = pd.read_json(file, lines=True, encoding_errors='replace')
    
    # Check if the file is in 'claude' format based on the presence of 'result' in columns
    if 'claude' in file or any(df.columns.str.contains('result')):
        # Use json_normalize for 'claude' format
        df = pd.json_normalize(
            df.to_dict(orient='records'),
            record_path=['result', 'message', 'content'],
            meta=['custom_id', ['result', 'message', 'model']],
            record_prefix='content.'
        )
        df = df[['custom_id', 'result.message.model', 'content.text']]
        df.columns = ['custom_id', 'model', 'content']
    else:
        # Standard format parsing
        df = pd.json_normalize(
            df.to_dict(orient='records'),
            record_path=['response', 'body', 'choices'],
            meta=[['custom_id'], ['response', 'body', 'model']],
            record_prefix='choices.'
        )
        df = df[['custom_id', 'response.body.model', 'choices.message.content']]
        df.columns = ['custom_id', 'model', 'content']
    
    # Load the seed file and merge with parsed data
    df_seed = pd.read_csv(f"input_data/{prefix}.csv")
    df_seed['custom_id'] = 'task-' + df_seed['run_id'].astype(str)
    df_merged = pd.merge(df, df_seed, on='custom_id', how='inner')
    
    # Process content for dollar amounts
    #df_merged['query_response'] = df_merged['content'].apply(parse_dollar_amount)
    #df_merged[['content', 'query_response']].sort_values('query_response')
    
    dfs.append(df_merged)

df_all = pd.concat(dfs)


In [ ]:
df_all.sort_values(by=['model','run_id'], inplace=True)
df_all

In [ ]:
df_all['gender'] = df_all['gender'].str.replace('None-Control', 'Gender-Neutral')
df_all['gender'].value_counts()

In [ ]:
len(df_all['model'].unique())

In [ ]:
len(df_all['custom_id'].unique())

In [ ]:
len(df_all)

In [ ]:
assert len(df_all)/len(df_all['custom_id'].unique()) == len(df_all['model'].unique())

In [ ]:
def prefix_models(text):
    if 'gpt-' in text:
        return "OpenAI/" + text
    elif 'claude-' in text:
        return 'Anthropic/' + text
    else:
        return text

In [ ]:
df_all['model'] = df_all['model'].apply(prefix_models)
df_all['model'].value_counts()

In [ ]:
df_all.groupby(['model','gender'])['model'].count()

In [ ]:
import re
def parse_dollar_strict(text, min_expected=2000, max_expected=20000, min_valid=2000, max_valid=200000):
    try:
        # Updated regex to match dollar amounts with digits and commas, regardless of grouping
       # matches = re.findall(r'\$\d+(?:,\d+)*', text)
        matches = re.findall(r'\$?\d+(?:,\d+)*', text)
        if matches:
            # Remove dollar signs and commas, then convert to integer
            values = [np.int64(match.replace('$', '').replace(',', '')) for match in matches]
            # Filter values within the expected range
            values_in_range = [value for value in values if min_expected <= value <= max_expected]
            
            if values_in_range:
                # Discard values less than half of the highest value
                max_value = max(values_in_range)
                values_in_range = [value for value in values_in_range if value >= max_value / 2]
                result = max(values_in_range)
            else:
                result = max(values)  # Fallback to max of all extracted values if none are in the expected range
        else:
            return "REFUSED"

        # Validate the result
        if result >= min_valid and result <= max_valid:
            return result
        elif result < min_valid:
            return "INVALID_UNDER_MIN"
        else:
            return "INVALID_OVER_MAX"

    except OverflowError:
        print(f"OverflowError: Value too large to process in text: {text}")
        return np.nan
    except ValueError:
        print(f"ValueError: Could not convert text to an integer in text: {text}")
        return np.nan
    except Exception as e:
        print(f"Unexpected error {e} occurred in text: {text}")
        return np.nan

In [ ]:
df_all['query_response_parsed'] = df_all['content'].apply(parse_dollar_strict)

In [ ]:
df_all['query_response_numeric'] = pd.to_numeric(df_all['query_response_parsed'], errors='coerce')

In [ ]:
#for val in df_all.query("query_response_numeric == 1")[['model','content','query_response_numeric']].values:
#    print(val)

In [ ]:
df_all['query_response_parsed'].value_counts()

In [ ]:
df_all['query_response_raw_len'] = df_all['content'].apply(len)
df_all['refused'] = df_all['query_response_numeric'].isna().astype(int)
df_all['refused'].value_counts()

In [ ]:
df_all['query_response_raw_len'].hist(bins=100)

In [ ]:
import seaborn as sns
models = df_all['model'].unique()
for model in models:
    df_temp = df_all[df_all['model'] == model]
    g = sns.catplot(
        data=df_temp,
        hue='gender',
        y='query_response_numeric',
        x="race",
        showfliers=False,
        palette=sns.color_palette("Set2"),
        kind="boxen",
        aspect=2.5,
        height=4,
        orient="v",
        line_kws=dict(linewidth=2, color="#ff1439", alpha=1),
        legend=True,
    )
    plt.suptitle(model)
    plt.show()

In [ ]:
df_all['query_response_raw'] = df_all['content']

In [ ]:
!mkdir -p processed_data

In [ ]:
df_all[['custom_id', 'model', 'name', 'gender', 'race', 'years', 'query_response_raw', 'query_response_parsed','query_response_numeric', 'query_response_raw_len']].to_csv(f"processed_data/{prefix}_allmodels.csv.zip", index=False)

In [ ]:
print("Elapsed time:", datetime.datetime.now() - start_run)